# Create Unstructured Amazon Bedrock Knowledge Base for Octank Financial


This notebook demonstrates how to create and configure an Amazon Bedrock Knowledge Base for unstructured data.

The Knowledge Base integrates Amazon S3 as the data source for the Octank Financial 10K document and uses Amazon OpenSearch Serverless as the vector store. It enables RAG by powering queries over unstructured financial and business content.

This unstructured knowledge base will be used in conjunction with the structured knowledge base to create agentic RAG using Strands Agents.


## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Amazon Bedrock foundation models enabled
* IAM role with permissions to create Amazon Bedrock Knowledge Base, Amazon S3 bucket, Amazon OpenSearch Serverless

Let's now install the requirement packages and define the needed clients to create our Amazon Bedrock Knowledge Base:


In [ ]:
import os
import json
import time
import uuid
import boto3
import logging
import requests
import botocore
from datetime import datetime


In [ ]:
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime")

print(f"AWS region: {region}")


In [ ]:
# Generate unique suffix for resource names
current_time = time.time()
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-4:]
suffix = f"{timestamp_str}"

print(f"Suffix: {suffix}")


## Step 1: Download Amazon Bedrock Knowledge Bases helper
To expedite the knowledge base configuration and creation we will be downloading the knowledge base utility file. This contains a helper to generate knowledge bases abstracting the multiple API calls that need to be used.


In [ ]:
url = "https://raw.githubusercontent.com/aws-samples/amazon-bedrock-samples/main/rag/knowledge-bases/features-examples/utils/knowledge_base.py"
target_path = "utils/knowledge_base.py"
os.makedirs(os.path.dirname(target_path), exist_ok=True)
response = requests.get(url)
with open(target_path, "w") as f:
    f.write(response.text)
print(f"Downloaded Knowledge Bases utils to {target_path}")


In [ ]:
from utils.knowledge_base import BedrockKnowledgeBase

## Step 2: Create Amazon Bedrock Knowledge Base for Unstructured Data
In this section we will configure the Amazon Bedrock Knowledge Base containing the Octank Financial 10K document. We will be using Amazon OpenSearch Serverless Service as the underlying vector store and Amazon S3 as the data source containing the PDF file.


In [ ]:
knowledge_base_name = f"octank-financial-unstructured-kb-{suffix}"
knowledge_base_description = "Octank Financial Unstructured Knowledge Base containing 10K financial document for business strategy and company information queries."
foundation_model = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

For this notebook, we'll create a Knowledge Base with an Amazon S3 data source containing the Octank Financial 10K PDF.


In [ ]:
data_bucket_name = f'octank-financial-unstructured-{suffix}-bucket'
data_sources = [{"type": "S3", "bucket_name": data_bucket_name}]


### Create the Amazon S3 bucket and upload the Octank Financial 10K document
We'll create an S3 bucket and upload the Octank Financial 10K PDF document that will serve as our unstructured data source.


In [ ]:
def create_s3_bucket(bucket_name, region=None):
    s3 = boto3.client('s3', region_name=region)

    try:
        if region is None or region == 'us-east-1':
            s3.create_bucket(Bucket=bucket_name)
        else:
            s3.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"Bucket '{bucket_name}' created successfully.")
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
            print(f"Bucket '{bucket_name}' already exists and is owned by you.")
        else:
            print(f"Failed to create bucket: {e.response['Error']['Message']}")

create_s3_bucket(data_bucket_name, region)


In [ ]:
def upload_directory(path, bucket_name):
    for root, dirs, files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root, file)
            print(f"Uploading file {file_to_upload} to {bucket_name}")
            s3_client.upload_file(file_to_upload, bucket_name, file)


In [ ]:
# Upload the Octank Financial 10K document
upload_directory("./sample_unstructured_data", data_bucket_name)


### Create the Unstructured Knowledge Base
We are now going to create the Knowledge Base using the abstraction located in the helper function we previously downloaded.


In [ ]:
unstructured_knowledge_base = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}',
    kb_description=knowledge_base_description,
    data_sources=data_sources,
    chunking_strategy="FIXED_SIZE", 
    suffix=f'{suffix}-u' 
)


### Start ingestion job
Once the KB and data source are created, we can start the ingestion job for the data source. During the ingestion job, KB will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database (OpenSearch Serverless).


In [ ]:
# Ensure that the kb is available
time.sleep(30)
# Sync knowledge base
unstructured_knowledge_base.start_ingestion_job()
# Keep the kb_id for invocation later in the invoke request
unstructured_kb_id = unstructured_knowledge_base.get_knowledge_base_id()
print(f"Unstructured Knowledge Base ID: {unstructured_kb_id}")


### Test the Unstructured Knowledge Base
We can now test the Knowledge Base to verify the Octank Financial 10K document has been ingested properly.


In [ ]:
query = "What is Octank Financial's primary business strategy?"

In [ ]:
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': unstructured_kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults": 5
                } 
            }
        }
    }
)

print("Response:")
print(response['output']['text'], end='\\n'*2)


### Store the Knowledge Base ID
Store the ID of the generated Unstructured Knowledge Base to use it in the main dual-knowledge-base RAG notebook.


In [ ]:
kb_region = region
%store unstructured_kb_id
%store kb_region
%store data_bucket_name

print("="*60)
print(f"Unstructured Knowledge Base ID: {unstructured_kb_id}")
print(f"Region: {kb_region}")
print(f"S3 Bucket: {data_bucket_name}")

print("="*60)
print("Configuration stored successfully!")



## Clean up the resources
**If you plan to use the other notebooks and continue with them, do not delete the Knowledge Base yet as it will be needed.**

When you are finished with the other notebooks, to avoid additional costs, delete the resources created.


In [ ]:
# Uncomment to delete resources when finished

# print("===============================Deleting Unstructured Knowledge Base and associated resources==============================\\n")
# unstructured_knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)
# print("Cleanup completed successfully!")



###  Summary
If all the above cells executed successfully, you have:

- Created an S3 bucket for unstructured data  
- Uploaded the Octank Financial 10K PDF  
- Created an Amazon Bedrock Knowledge Base  
- Configured OpenSearch Serverless as the vector store  
- Successfully ingested the document  
- Tested a query with knowledge base's `RetrieveAndGenerate` API
- Stored the Knowledge Base ID for use in the main notebook  

You are now ready to proceed to the main `2-unstructured-structured-rag-agent.ipynb` notebook!
